In [8]:
!pwd

/mydata/cookiemonster/data/patcg


In [31]:
import pandas as pd

impressions = pd.read_csv(
    "/mydata/cookiemonster/data/patcg/smaller_data/publishers/renamed_filtered_impressions.csv"
)
conversions = pd.read_csv(
    "/mydata/cookiemonster/data/patcg/smaller_data/advertisers/renamed_conversions.csv"
)

In [34]:
impressions

,device_id,exp_timestamp,exp_attribute_2
0,547842,1.654247e+09,8
1,547842,1.654336e+09,3
2,547842,1.654500e+09,7
3,547842,1.654631e+09,4
4,547842,1.654836e+09,8
...,...,...,...
96685,15962507,1.655883e+09,3
96686,15962507,1.656144e+09,2
96687,15962508,1.654979e+09,3
96688,15962508,1.656027e+09,9


In [38]:
conversions.shape

(24240774, 4)

In [39]:
i = impressions.drop_duplicates("device_id", keep="first")
x = i.merge(conversions, how="inner", on="device_id")
y = x[x["conv_timestamp"] >= x["exp_timestamp"]]
y

,device_id,exp_timestamp,exp_attribute_2,conv_timestamp,conv_attribute_2,conv_amount
0,547842,1.654247e+09,8,1.656546e+09,2,23.09
1,547843,1.654101e+09,9,1.654132e+09,5,5.58
2,547843,1.654101e+09,9,1.655062e+09,1,5.76
3,547843,1.654101e+09,9,1.656136e+09,4,4.10
4,547844,1.654486e+09,7,1.656039e+09,2,2.66
...,...,...,...,...,...,...
45037,15962506,1.654064e+09,1,1.655333e+09,9,18.78
45038,15962506,1.654064e+09,1,1.655957e+09,4,4.43
45039,15962507,1.654362e+09,8,1.655558e+09,3,39.69
45040,15962507,1.654362e+09,8,1.655885e+09,9,15.83


In [52]:
import math

A = 0.05
B = 0.01
CAP_VALUE = 15

SCHEDULED_BATCH_SIZE = 18682297
DIMENSION_DOMAIN_SIZE = 10


def set_epsilon_given_accuracy(a, b, s, n):
    return s * math.log(1 / b) / (n * a)  # 0.069

In [46]:
impressions.rename(columns={"exp_attribute_6": "ad_creative"}, inplace=True)
conversions.rename(columns={"conv_amount": "amount"}, inplace=True)
impressions = impressions[["exp_timestamp", "device_id", "ad_creative"]]
conversions = conversions[["conv_timestamp", "device_id", "amount"]]

In [47]:
conversions = conversions.sort_values(["conv_timestamp"]).reset_index()

In [48]:
conversions

,index,conv_timestamp,device_id,amount
0,6650142,1.654042e+09,e8f5cfce-2f16-46f0-b9c2-79894f263963,6.61
1,42194683,1.654042e+09,78a97efb-431d-4ab4-8223-82a0d97edd91,4.66
2,9775651,1.654042e+09,439b5233-8526-446c-b00a-285029fe88b2,23.58
3,59300100,1.654042e+09,dac08298-07c7-420a-bddf-5924ab51f1cb,13.15
4,1103134,1.654042e+09,1b8d4370-33e7-4314-a3a3-0ebddc87a1b3,17.61
...,...,...,...,...
74729186,53016753,1.656634e+09,3d213b38-4a62-419f-abb6-f52d1cca0054,9.56
74729187,53765417,1.656634e+09,2ba779a1-8af3-4461-b6a0-30ad545d8f6c,10.63
74729188,69953473,1.656634e+09,09c6c99b-20ce-42c3-be3c-a4c9b6f47789,6.23
74729189,33909227,1.656634e+09,5ce22836-c50c-402b-b424-161ac2dc0aec,10.14


In [51]:
# Enforce Cap on user contribution

seen_users = set()
row_count = 0


def __mark_include(row: pd.Series):
    global seen_users
    global row_count
    if row_count == SCHEDULED_BATCH_SIZE:
        row_count = 0
        seen_users = set()

    user = row["device_id"]
    if user in seen_users:
        return False
    else:
        seen_users.add(user)
        row_count += 1
        return True


conversions = conversions.assign(
    include=conversions.apply(
        lambda row: __mark_include(row),
        axis=1,
    ),
)

In [54]:
conversions.groupby(["include"]).size()

include
False     7649375
True     67079816
dtype: int64

In [55]:
conversions = conversions.loc[conversions.include]
conversions = conversions.drop(columns=["include"])

In [64]:
conversions

,conv_timestamp,device_id,batch_id,epsilon,amount
0,1.654042e+09,e8f5cfce-2f16-46f0-b9c2-79894f263963,_0,0.073950,6.61
1,1.654042e+09,78a97efb-431d-4ab4-8223-82a0d97edd91,_0,0.073950,4.66
2,1.654042e+09,439b5233-8526-446c-b00a-285029fe88b2,_0,0.073950,23.58
3,1.654042e+09,dac08298-07c7-420a-bddf-5924ab51f1cb,_0,0.073950,13.15
4,1.654042e+09,1b8d4370-33e7-4314-a3a3-0ebddc87a1b3,_0,0.073950,17.61
...,...,...,...,...,...
67079811,1.656634e+09,3d213b38-4a62-419f-abb6-f52d1cca0054,_3,0.125221,9.56
67079812,1.656634e+09,2ba779a1-8af3-4461-b6a0-30ad545d8f6c,_3,0.125221,10.63
67079813,1.656634e+09,09c6c99b-20ce-42c3-be3c-a4c9b6f47789,_3,0.125221,6.23
67079814,1.656634e+09,5ce22836-c50c-402b-b424-161ac2dc0aec,_3,0.125221,10.14


In [58]:
conversions = conversions.reset_index()

In [60]:
# Tag each batch with its batch id

scheduled_batch_size = SCHEDULED_BATCH_SIZE

num_conversions = len(conversions)
megabatches_per_query = num_conversions // SCHEDULED_BATCH_SIZE
print("megabatches", megabatches_per_query)
remaining = num_conversions % SCHEDULED_BATCH_SIZE
print("minibatch size", remaining)

for i in range(megabatches_per_query):
    batch_start = i * SCHEDULED_BATCH_SIZE
    batch_end = (i + 1) * SCHEDULED_BATCH_SIZE
    print("   ", batch_start, batch_end)

    conversions.loc[batch_start:batch_end, "batch_id"] = f"_{i}"

if remaining > 0:
    batch_start = megabatches_per_query * SCHEDULED_BATCH_SIZE
    batch_end = (megabatches_per_query * SCHEDULED_BATCH_SIZE) + remaining
    print("  rem   ", batch_start, batch_end)
    conversions.loc[batch_start:batch_end, "batch_id"] = f"_{megabatches_per_query}"

conversions

megabatches 3
minibatch size 11032925
    0 18682297
    18682297 37364594
    37364594 56046891
  rem    56046891 67079816


,level_0,index,conv_timestamp,device_id,amount,batch_id
0,0,6650142,1.654042e+09,e8f5cfce-2f16-46f0-b9c2-79894f263963,6.61,_0
1,1,42194683,1.654042e+09,78a97efb-431d-4ab4-8223-82a0d97edd91,4.66,_0
2,2,9775651,1.654042e+09,439b5233-8526-446c-b00a-285029fe88b2,23.58,_0
3,3,59300100,1.654042e+09,dac08298-07c7-420a-bddf-5924ab51f1cb,13.15,_0
4,4,1103134,1.654042e+09,1b8d4370-33e7-4314-a3a3-0ebddc87a1b3,17.61,_0
...,...,...,...,...,...,...
67079811,74729186,53016753,1.656634e+09,3d213b38-4a62-419f-abb6-f52d1cca0054,9.56,_3
67079812,74729187,53765417,1.656634e+09,2ba779a1-8af3-4461-b6a0-30ad545d8f6c,10.63,_3
67079813,74729188,69953473,1.656634e+09,09c6c99b-20ce-42c3-be3c-a4c9b6f47789,6.23,_3
67079814,74729189,33909227,1.656634e+09,5ce22836-c50c-402b-b424-161ac2dc0aec,10.14,_3


In [61]:
# Set epsilon from accuracy

batch_sizes = conversions.groupby("batch_id").size().reset_index(name="batch_size")
batch_sizes = batch_sizes.assign(
    nonempty_batch_size=batch_sizes.apply(
        lambda row: (0.01 * row.batch_size) / 10, axis=1
    )
)
batch_sizes = batch_sizes.assign(
    epsilon=batch_sizes.apply(
        lambda row: set_epsilon_given_accuracy(
            A, B, CAP_VALUE, row.nonempty_batch_size
        ),
        axis=1,
    )
)
batch_sizes

,batch_id,batch_size,nonempty_batch_size,epsilon
0,_0,18682297,18682.297,0.073950
1,_1,18682297,18682.297,0.073950
2,_2,18682297,18682.297,0.073950
3,_3,11032925,11032.925,0.125221


In [62]:
conversions = batch_sizes.merge(conversions, how="inner", on="batch_id")
conversions

,batch_id,batch_size,nonempty_batch_size,epsilon,level_0,index,conv_timestamp,device_id,amount
0,_0,18682297,18682.297,0.073950,0,6650142,1.654042e+09,e8f5cfce-2f16-46f0-b9c2-79894f263963,6.61
1,_0,18682297,18682.297,0.073950,1,42194683,1.654042e+09,78a97efb-431d-4ab4-8223-82a0d97edd91,4.66
2,_0,18682297,18682.297,0.073950,2,9775651,1.654042e+09,439b5233-8526-446c-b00a-285029fe88b2,23.58
3,_0,18682297,18682.297,0.073950,3,59300100,1.654042e+09,dac08298-07c7-420a-bddf-5924ab51f1cb,13.15
4,_0,18682297,18682.297,0.073950,4,1103134,1.654042e+09,1b8d4370-33e7-4314-a3a3-0ebddc87a1b3,17.61
...,...,...,...,...,...,...,...,...,...
67079811,_3,11032925,11032.925,0.125221,74729186,53016753,1.656634e+09,3d213b38-4a62-419f-abb6-f52d1cca0054,9.56
67079812,_3,11032925,11032.925,0.125221,74729187,53765417,1.656634e+09,2ba779a1-8af3-4461-b6a0-30ad545d8f6c,10.63
67079813,_3,11032925,11032.925,0.125221,74729188,69953473,1.656634e+09,09c6c99b-20ce-42c3-be3c-a4c9b6f47789,6.23
67079814,_3,11032925,11032.925,0.125221,74729189,33909227,1.656634e+09,5ce22836-c50c-402b-b424-161ac2dc0aec,10.14


In [63]:
conversions = conversions[
    ["conv_timestamp", "device_id", "batch_id", "epsilon", "amount"]
]

In [67]:
# Repeat conversions once per query and tag the different query types with their query type id

all_conversions = pd.concat([conversions] * DIMENSION_DOMAIN_SIZE, ignore_index=True)

for i in range(DIMENSION_DOMAIN_SIZE):
    query_start = i * num_conversions
    query_end = (i + 1) * num_conversions
    print(query_start, query_end)
    all_conversions.loc[query_start:query_end, "filter"] = str(i)
all_conversions

0 67079816
67079816 134159632
134159632 201239448
201239448 268319264
268319264 335399080
335399080 402478896
402478896 469558712
469558712 536638528
536638528 603718344
603718344 670798160


,conv_timestamp,device_id,batch_id,epsilon,amount,filter
0,1.654042e+09,e8f5cfce-2f16-46f0-b9c2-79894f263963,_0,0.073950,6.61,0
1,1.654042e+09,78a97efb-431d-4ab4-8223-82a0d97edd91,_0,0.073950,4.66,0
2,1.654042e+09,439b5233-8526-446c-b00a-285029fe88b2,_0,0.073950,23.58,0
3,1.654042e+09,dac08298-07c7-420a-bddf-5924ab51f1cb,_0,0.073950,13.15,0
4,1.654042e+09,1b8d4370-33e7-4314-a3a3-0ebddc87a1b3,_0,0.073950,17.61,0
...,...,...,...,...,...,...
670798155,1.656634e+09,3d213b38-4a62-419f-abb6-f52d1cca0054,_3,0.125221,9.56,9
670798156,1.656634e+09,2ba779a1-8af3-4461-b6a0-30ad545d8f6c,_3,0.125221,10.63,9
670798157,1.656634e+09,09c6c99b-20ce-42c3-be3c-a4c9b6f47789,_3,0.125221,6.23,9
670798158,1.656634e+09,5ce22836-c50c-402b-b424-161ac2dc0aec,_3,0.125221,10.14,9


In [ ]:
all_conversions["query_key"] = (
    "ad" + all_conversions["filter"] + "_" + all_conversions["batch_id"]
)
all_conversions